### Load standard libraries


In [3]:
import sys
print("System version\n",sys.version)

System version
 3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]


In [1]:
import pickle
import numpy as np
import pandas as pd
import csv
import json
import os
import random
import shutil
from tqdm import tqdm
import time

from os import listdir, makedirs
from os.path import join, exists, expanduser

import tensorflow as tf
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

from keras_tqdm import TQDMNotebookCallback

import matplotlib.pyplot as plt
from numpy.random import seed
from keras.utils import plot_model

from keras import layers
from keras import models
from keras.callbacks import EarlyStopping

import onnxmltools
import keras2onnx

import sklearn.datasets
import sklearn.model_selection
import keras
from sklearn.metrics import accuracy_score

import cv2

Using TensorFlow backend.


### Run general function notebook

In [2]:
%run ./general_functions.ipynb

In [3]:
#Check the hardware for the training
from tensorflow.python.client import device_lib
print("\nYour Device List:\n",device_lib.list_local_devices())


Your Device List:
 [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17184768617109123558
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6700198133
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3401646839610310401
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [ ]:
#Set the global seed for all the experiments
seed(4292)

### Load and prepare the dataset

In [9]:
def prepareConstants(img_width = None ,img_height = None,img_size = None,batch_size = None):
    
    if(img_width is None):
        img_width = 224
    if(img_height is None):
        img_height = 224
    if(img_size is None):
        img_size = 224
    if(batch_size is None):
        batch_size = 16
    
    print("\nImage Width:",img_width)
    print("Image Height:",img_height)
    print("Image Size:",img_size)
    print("Batch Size:",batch_size)
    
    return img_width, img_height, img_size, batch_size

In [13]:
def setDirs(train_data_dir = None, validation_data_dir = None):
    
    if(train_data_dir is None):
        train_data_dir = '../fruits-360/Training/'
    
    if(validation_data_dir is None):
        validation_data_dir = '../fruits-360/Test/'
    
    print("\nTrain dir:",train_data_dir)
    print("Test dir:",validation_data_dir)
    
    return train_data_dir, validation_data_dir

In [1]:
def prepareDatasets(train_data_dir,validation_data_dir,img_height,img_width,batch_size, augmentation = False, test_train_aug = True):
    
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
    
    test_datagen = ImageDataGenerator(rescale=1. / 255) 

    print("\nImport Trainingset")
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

    print("\nImport Validationset")
    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle = False,
        class_mode='categorical')
    
    if(augmentation is True):
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        
        if(test_train_aug is True):
            datagen.fit(train_generator)
            datagen.fit(validation_generator)
            print("\nTrain and test datasets augmented")
        elif(test_train_aug is False):
            datagen.fit(train_generator)
            print("\nTrain dataset augmented")
    
    return train_generator, validation_generator

SyntaxError: invalid syntax (<ipython-input-1-c276909f0303>, line 24)

In [18]:
def checkData(train_generator = None,validation_generator = None):
    
    if(train_generator is not None):
        nb_train_samples = train_generator.n
        print("\nTrain Samples: ",nb_train_samples)
        train_classes = max(train_generator.classes + 1 )
        print("Train Classes:", train_classes)
        print("Class Indexes:\n",train_generator.class_indices)
        
    if(validation_generator is not None):
        nb_validation_samples = validation_generator.n
        print("\nValidation Samples: ",nb_validation_samples)
        validation_classes = max(validation_generator.classes + 1 )
        print("\nTrain Classes:", validation_classes)
        print("\nClass Indexes:\n",validation_generator.class_indices)
        
    return nb_train_samples, train_classes, nb_validation_samples, validation_classes
        